In [17]:
import pandas as pd
import numpy as np
import random
import tkinter as tk
from tkinter import messagebox, Canvas
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Charger les fichiers CSV
gagnants = pd.read_csv("dataset_fanorona/fanorona_winning_positions_cleaned.csv", header=None)  # Remplace avec le bon chemin
perdants = pd.read_csv("dataset_fanorona/fanorona_losing_positions_cleaned.csv", header=None)

# Ajouter la colonne cible (1 pour gagnant, -1 pour perdant)
gagnants["score"] = 1
perdants["score"] = -1

# Fusionner les données
data = pd.concat([gagnants, perdants], ignore_index=True)

# Séparer les features et labels
X = data.iloc[:, :-1].values  # Les configurations de plateau
y = data["score"].values  # Les scores

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner l'arbre de décision
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Évaluer le modèle
y_pred = clf.predict(X_test)
print("Précision du modèle :", accuracy_score(y_test, y_pred))

# Classe du jeu Fanorona Telo
class FanoronaTelo:
    def __init__(self):
        self.board = [0] * 9  # 0 = vide, 1 = blanc, -1 = noir
        self.current_player = 1
        self.placed_pieces = {1: 0, -1: 0}  # Nombre de pièces placées par joueur
        self.phase = "placement"  # Phase du jeu : "placement" ou "déplacement"
        self.valid_lines = [
            [0, 1, 2], [0, 3, 6], [0, 4, 8], [3, 4, 5],
            [6, 7, 8], [2, 4, 6], [1, 4, 7], [2, 5, 8]
        ]
        self.positions = [
            (100, 50), (200, 50), (300, 50),
            (100, 150), (200, 150), (300, 150),
            (100, 250), (200, 250), (300, 250)
        ]
        self.lines = [
            (0, 1), (1, 2), (0, 3), (3, 6), (0, 4), (4, 8),
            (3, 4), (4, 5), (6, 7), (7, 8), (2, 4), (4, 6),
            (1, 4), (4, 7), (2, 5), (5, 8)
        ]
    
    def get_valid_moves(self):
        if self.phase == "placement":
            return [i for i in range(9) if self.board[i] == 0]
        else:
            moves = []
            for line in self.valid_lines:
                for i in range(len(line)):
                    if self.board[line[i]] == self.current_player:
                        for j in range(len(line)):
                            if self.board[line[j]] == 0:
                                moves.append((line[i], line[j]))  # (position actuelle, nouvelle position)
            return moves
    
    def make_move(self, move):
        if self.phase == "placement":
            if self.board[move] == 0:
                self.board[move] = self.current_player
                self.placed_pieces[self.current_player] += 1
                if self.placed_pieces[1] == 3 and self.placed_pieces[-1] == 3:
                    self.phase = "déplacement"
                self.current_player *= -1
                return True
        else:
            src, dst = move
            if (self.board[src] == self.current_player and self.board[dst] == 0 and
                any(src in line and dst in line for line in self.valid_lines)):
                self.board[src] = 0
                self.board[dst] = self.current_player
                self.current_player *= -1
                return True
        return False
    
    def get_state(self):
        return self.board.copy()

# Fonction IA pour choisir le meilleur coup
def choose_best_move(game):
    best_move = None
    best_score = -float('inf') if game.current_player == 1 else float('inf')
    
    for move in game.get_valid_moves():
        temp_board = game.get_state()
        if game.phase == "placement":
            temp_board[move] = game.current_player
        else:
            src, dst = move
            temp_board[src] = 0
            temp_board[dst] = game.current_player
        
        score = clf.predict([temp_board])[0]
        
        if (game.current_player == 1 and score > best_score) or (game.current_player == -1 and score < best_score):
            best_score = score
            best_move = move
    
    return best_move

# Interface graphique
class FanoronaGUI:
    def __init__(self, root):
        self.game = FanoronaTelo()
        self.root = root
        self.root.title("Fanorona Telo")
        self.canvas = Canvas(root, width=400, height=300)
        self.canvas.pack()
        self.selected = None
        self.create_board()
        self.update_board()
        self.canvas.bind("<Button-1>", self.on_click)
    
    def create_board(self):
        for line in self.game.lines:
            x1, y1 = self.game.positions[line[0]]
            x2, y2 = self.game.positions[line[1]]
            self.canvas.create_line(x1, y1, x2, y2, fill="black", width=2)
    
    def update_board(self):
        self.canvas.delete("pieces")
        for i, (x, y) in enumerate(self.game.positions):
            color = ""
            if self.game.board[i] == 1:
                color = "white"
            elif self.game.board[i] == -1:
                color = "black"
            if color:
                self.canvas.create_oval(x-15, y-15, x+15, y+15, fill=color, tags="pieces")
    
    def on_click(self, event):
        for i, (x, y) in enumerate(self.game.positions):
            if (x-20 <= event.x <= x+20) and (y-20 <= event.y <= y+20):
                if self.game.make_move(i):
                    self.update_board()
                    self.root.after(500, self.ai_play)
                break
    
    def ai_play(self):
        move = choose_best_move(self.game)
        if move is not None:
            self.game.make_move(move)
            self.update_board()

# Lancer l'interface
tk_root = tk.Tk()
app = FanoronaGUI(tk_root)
tk_root.mainloop()


Précision du modèle : 0.9508196721311475


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\miran\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\miran\AppData\Local\Temp\ipykernel_4488\787133875.py", line 148, in on_click
    if self.game.make_move(i):
       ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miran\AppData\Local\Temp\ipykernel_4488\787133875.py", line 81, in make_move
    src, dst = move
    ^^^^^^^^
TypeError: cannot unpack non-iterable int object
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\miran\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\miran\AppData\Local\Temp\ipykernel_4488\787133875.py", line 148, in on_click
    if self.game.make_move(i):
       ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miran\AppData\Local\Temp\ipykernel_4488\787133875.py", line 81, in make_move
    src, dst =